In [1]:
%load_ext autoreload
%autoreload 2

In [32]:
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from adptSeg.adaptation.model_wrapper import FeatureType, ModelFeaturesExtractor
from fundseg.utils.checkpoints import load_model_from_checkpoints

import wandb

In [3]:
api = wandb.Api()
runs = api.runs("liv4d-polytechnique/Probing-Lesions-Segmentation-Positions")


In [43]:
def get_confMat(df):
    pass

results = []
for r in runs:
    ftype = r.config['feature_type']
    pos = r.config['position']
    table = r.logged_artifacts()[0]
    table.download()

    table_name = table.name
    avg_table_path = r.summary['conf_mat_table']['path']
    table_path = f"artifacts/{table_name}/conf_mat_table.table.json"
    with open(table_path) as f:
        data = json.load(f)
    df = pd.DataFrame(data["data"], columns=data["columns"])
    datasets = df['Actual'].unique()
    datasets.sort()
    confMat = np.zeros((len(datasets), len(datasets)))
    for i, dataset in enumerate(datasets):
        for j, dataset2 in enumerate(datasets):
            confMat[i, j] = df[(df['Actual'] == dataset) & (df['Predicted'] == dataset2)]['nPredictions'].values[0]

    
    results.append({
        'feature_type': ftype,
        'position': pos,
        'confMat': confMat,
        'datasets': datasets
    })

results.sort(key=lambda x: x['position'])
results.sort(key=lambda x: x['feature_type'])


wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


In [47]:
for result in results:
    print(result['feature_type'], result['position'])
    cmat = result['confMat']
    accu = np.sum(np.diag(cmat)) / np.sum(cmat)
    print(accu, )
    

DECODER 0
0.607
DECODER 1
0.871
DECODER 2
0.881
DECODER 3
0.661
DECODER 4
0.49
ENCODER 0
0.356
ENCODER 1
0.563
ENCODER 2
0.859
ENCODER 3
0.923
ENCODER 4
0.925


In [55]:

model = load_model_from_checkpoints(root_directory='../../')

Test dataset: IDRID
Test dataset: FGADR
Test dataset: MESSIDOR
Test dataset: DDR
Test dataset: RETLES


In [66]:
featureExtractor = ModelFeaturesExtractor(model, position=4, feature_type=FeatureType.DECODER)
featureExtractor.out_chans

16

3